# BioExplorer - Neurons
![](../bioexplorer_neurons_banner.png)

# Circuit

In [ ]:
import bluepy
import os

visualization_folder = os.getenv("VISUALIZATION_FOLDER")
circuit_config = '%s/simulations/vizCa2p0_1x7/BlueConfig' % visualization_folder
circuit = bluepy.Circuit(circuit_config)

In [ ]:
cells = circuit.cells.get()

In [ ]:
import os
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from tqdm.notebook import tqdm

db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_schema = 'o1'

db_connection_string = 'postgresql+psycopg2://%s:%s@%s:5432/%s' % (db_user, db_password, db_host, db_name)
print('Connection string: ' + db_connection_string + ', schema: ' + db_schema)

engine = create_engine(db_connection_string)
conn = engine.connect()

In [ ]:
cells

In [ ]:
# Etypes
print('Deleting existing etype')
with Session(engine) as session:
    session.execute('DELETE FROM %s.electrical_type' % db_schema)
    session.commit()

etypes = circuit.cells.etypes
etypes_guids = dict()
guid = 0
with Session(engine) as session:
    for etype in etypes:
        session.execute(
            'INSERT INTO %s.electrical_type VALUES (:guid, :etype)' % db_schema,
            {
                'guid': guid,
                'etype': etype
            }
        )
        etypes_guids[etype] = guid
        guid += 1
    session.commit()

In [ ]:
# Etypes
print('Deleting existing mtype')
with Session(engine) as session:
    session.execute('DELETE FROM %s.morphological_type' % db_schema)
    session.commit()

mtypes = circuit.cells.mtypes
mtypes_guids = dict()
guid = 0
with Session(engine) as session:
    for mtype in mtypes:
        session.execute(
            'INSERT INTO %s.morphological_type VALUES (:guid, :mtype)' % db_schema,
            {
                'guid': guid,
                'mtype': mtype
            }
        )
        mtypes_guids[mtype] = guid
        guid += 1
    session.commit()

In [ ]:
# Layers
print('Deleting existing layer')
with Session(engine) as session:
    session.execute('DELETE FROM %s.layer' % db_schema)
    session.commit()

layers = set(cells['layer'])
layers_guids = dict()
guid = 0
with Session(engine) as session:
    for layer in layers:
        session.execute(
            'INSERT INTO %s.layer VALUES (:guid, :layer)' % db_schema,
            {
                'guid': guid,
                'layer': layer
            }
        )
        layers_guids[layer] = guid
        guid += 1
    session.commit()

In [ ]:
# Morphology class
print('Deleting existing morphology classes')
with Session(engine) as session:
    session.execute('DELETE FROM %s.morphology_class' % db_schema)
    session.commit()

morphology_classes = set(cells.morph_class)
morphology_classes_guids = dict()
guid = 0
with Session(engine) as session:
    for morphology_class in morphology_classes:
        session.execute(
            'INSERT INTO %s.morphology_class VALUES (:guid, :morphology_class)' % db_schema,
            {
                'guid': guid,
                'morphology_class': morphology_class
            }
        )
        morphology_classes_guids[morphology_class] = guid
        guid += 1
    session.commit()

In [ ]:
# Synapse class
print('Deleting existing Synapse classes')
with Session(engine) as session:
    session.execute('DELETE FROM %s.synapse_class' % db_schema)
    session.commit()

synapse_classes = set(cells.synapse_class)
synapse_classes_guids = dict()
guid = 0
with Session(engine) as session:
    for synapse_class in synapse_classes:
        session.execute(
            'INSERT INTO %s.synapse_class VALUES (:guid, :synapse_class)' % db_schema,
            {
                'guid': guid,
                'synapse_class': synapse_class
            }
        )
        synapse_classes_guids[synapse_class] = guid
        guid += 1
    session.commit()

In [ ]:
# Regions (minicolumns)
print('Deleting existing regions')
with Session(engine) as session:
    session.execute('DELETE FROM %s.region' % db_schema)
    session.commit()

regions = set(cells.minicolumn)
regions_guids = dict()
guid = 0
with Session(engine) as session:
    for region in regions:
        session.execute(
            'INSERT INTO %s.region VALUES (:guid, :region)' % db_schema,
            {
                'guid': guid,
                'region': region
            }
        )
        regions_guids[region] = guid
        guid += 1
    session.commit()

In [ ]:
# Targets
print('Deleting existing targets')
with Session(engine) as session:
    session.execute('DELETE FROM %s.target' % db_schema)
    session.commit()

targets = set(circuit.cells.targets)
targets_guids = dict()
guid = 0
with Session(engine) as session:
    for target in targets:
        session.execute(
            'INSERT INTO %s.target VALUES (:guid, :target)' % db_schema,
            {
                'guid': guid,
                'target': target
            }
        )
        targets_guids[target] = guid
        guid += 1
    session.commit()

In [ ]:
# Morphologies
morphologies_guids = dict()
print('Deleting existing morphologies')
with Session(engine) as session:
    session.execute('DELETE FROM %s.morphology' % db_schema)
    session.commit()

morphologies = set(cells['morphology'])
morphologies_guids = dict()

guid = 0
with Session(engine) as session:
    for morphology in morphologies:
        try:
            session.execute(
                'INSERT INTO %s.morphology VALUES (:guid, :basename)' % db_schema,
                {
                    'guid': guid,
                    'basename': morphology
                }
            )
            morphologies_guids[morphology] = guid
            guid += 1
            session.commit()
        except Exception as e:
            print(e)
        session.commit()

In [ ]:
# Nodes
import quaternion

print('Deleting existing nodes')
with Session(engine) as session:
    session.execute('DELETE FROM %s.node' % db_schema)
    session.commit()

guid = 0
print('Populating nodes')
with Session(engine) as session:
    for cell in cells.values:
        try:
               
            morphology = morphologies_guids[cell[0]]
            region = regions_guids[cell[2]]
            layer = layers_guids[cell[3]]
            mtype = mtypes_guids[cell[4]]
            morphology_class = morphology_classes_guids[cell[5]]
            synapse_class = synapse_classes_guids[cell[6]]
            etype = etypes_guids[cell[7]]

            orientation = cell[12]
            q = quaternion.from_rotation_matrix(
                [
                    [orientation[0][0], orientation[0][1], orientation[0][2]],
                    [orientation[1][0], orientation[1][1], orientation[1][2]],
                    [orientation[2][0], orientation[2][1], orientation[2][2]]
                ]
            )
            session.execute(
                'INSERT INTO %s.node VALUES (:guid, :x, :y, :z, :rotation_x, :rotation_y, ' \
                    ':rotation_z, :rotation_w, :morphology_guid, :morphology_class_guid, :electrical_type_guid, ' \
                    ':morphological_type_guid, :region_guid, :layer_guid, :synapse_class_guid)' % db_schema,
                {
                    'guid': guid,
                    'x': float(cell[9]),
                    'y': float(cell[10]),
                    'z': float(cell[11]),
                    'rotation_x': float(q.x),
                    'rotation_y': float(q.y),
                    'rotation_z': float(q.z),
                    'rotation_w': float(q.w),
                    'morphology_guid': morphology,
                    'morphology_class_guid': morphology_class,
                    'electrical_type_guid': etype,
                    'morphological_type_guid': mtype,
                    'region_guid': region,
                    'layer_guid': layer,
                    'synapse_class_guid': synapse_class
                }
            )
            guid += 1
            if guid % 1000 == 0:
                session.commit()
        except Exception as e:
            print(e)
            break
        session.commit()

In [ ]:
# Target <-> Node
morphologies_guids = dict()
print('Deleting existing target <-> node')
with Session(engine) as session:
    session.execute('DELETE FROM %s.target_node' % db_schema)
    session.commit()

targets = set(circuit.cells.targets)

with Session(engine) as session:
    for target in targets:
        for guid in circuit.cells.ids(group=target):
            try:
                session.execute(
                    'INSERT INTO %s.target_node VALUES (:node_guid, :target_guid)' % db_schema,
                    {
                        'node_guid': int(guid),
                        'target_guid': int(targets_guids[target])
                    }
                )
            except Exception as e:
                print(e)
    session.commit()

## Morphology Sections

In [ ]:
# Section types
section_types = ['undefined', 'Soma', 'Axon', 'Basal dendrite', 'Apical dendrite']

print('Deleting existing section type')
with Session(engine) as session:
    session.execute('DELETE FROM %s.section_type' % db_schema)
    session.commit()

guid = 0
with Session(engine) as session:
    for section_type in section_types:
        session.execute(
            'INSERT INTO %s.section_type VALUES (:guid, :code)' % db_schema,
            {
                
                
                
                
                
                
                
                
                
                
                
                
                'guid': guid,
                'code': section_types[guid]
            }
        )
        guid += 1
    session.commit()

In [ ]:
import morphio
import numpy as np

morphology_folder = circuit.config['morphologies']

with Session(engine) as session:
    session.execute('DELETE FROM %s.section' % db_schema)
    session.commit()

count = 0
with Session(engine) as session:
    for morphology in morphologies_guids:
        morphology_file = os.path.join(morphology_folder, morphology + '.h5')
        m = morphio.Morphology(morphology_file)

        for section in m.sections:
            section_type_guid = section.type
            parent = -1
            if not section.is_root:
                parent =  section.parent.id

            points = list()
            for i in range(len(section.points)):
                point = section.points[i]
                points.append(point[0])
                points.append(point[1])
                points.append(point[2])
                points.append(section.diameters[i])
            buffer = np.array(points, dtype=np.float32).tobytes()

            point = section.points[0]

            session.execute(
                'INSERT INTO %s.section VALUES (:morphology_guid, :section_guid, :section_parent_guid, :section_type_guid, :points, :x, :y, :z)' % db_schema,
                {
                    'morphology_guid': int(morphologies_guids[morphology]),
                    'section_guid': int(section.id),
                    'section_parent_guid': int(parent),
                    'section_type_guid': int(section_type_guid),
                    'points': buffer,
                    'x': float(point[0]),
                    'y': float(point[1]),
                    'z': float(point[2])
                }
            )
        count += 1
        if count % 100 == 0:
            print('%d/%d' % (count, len(morphologies_guids)))
            session.commit()
    session.commit()

# Synapses

In [ ]:
import h5py
import glob

nrn_position_files = glob.glob('/tmp/nrn_positions.h5.*')
nrn_position_files.sort()

In [ ]:
guid = 0
error = False

with Session(engine) as session:
    session.execute('DELETE FROM %s.synapse WHERE guid>=%i' % (db_schema, guid))
    session.commit()

with Session(engine) as session:
    for i in tqdm(range(len(nrn_position_files))):
        nrn_positions = h5py.File(nrn_position_files[i])
        for key in nrn_positions.keys():
            try:
                if key=='info':
                    continue
                preNeuron = int(key[1:])
                data = nrn_positions[key]
                for d in data:
                    session.execute(
                        'INSERT INTO %s.synapse VALUES (:guid, :preNeuron, :postNeuron, :sx, :sy, :sz, :cx, :cy, :cz)' % db_schema,
                        {
                            'guid': guid,
                            'preNeuron': preNeuron - 1,
                            'postNeuron': int(d[0]) - 1,
                            'sx': float(d[1]),
                            'sy': float(d[2]),
                            'sz': float(d[3]),
                            'cx': float(d[4]),
                            'cy': float(d[5]),
                            'cz': float(d[6]),
                        }
                    )
                    guid += 1
                # session.commit()
            except Exception as e:
                print(preNeuron)
                print(e)
                error = True
                break
        session.commit()

# Reports (To be done)

## Spikes

In [ ]:
conn.rollback()
cursor = conn.cursor()
sql_command = "DELETE FROM " + PGSCHEMA + ".spikes_report";
cursor.execute(sql_command, conn)
conn.commit()

spike_report = '$VISUALIZATION_FOLDER/simulations/vizCa2p0_1x7/out.dat'
with open(spike_report) as f:
    line = f.readline()
    cursor = conn.cursor()
    count = 0
    while line:
        values = line.split()
        if len(values) == 2:
            sql_command = "INSERT INTO " + PGSCHEMA + ".spikes_report VALUES (%f, %f);" % (int(values[1]) - 1, float(values[0]))
            cursor.execute(sql_command, conn)

            count += 1
            if count % 1000 == 0:
                conn.commit()
            
        line = f.readline()            
            
    conn.commit()

## Compartments

In [ ]:
import pybinreports
compartment_report = pybinreports.BinReport(
    '$VISUALIZATION_FOLDER/simulations/vizCa2p0_1x7/compartments.bbp')

In [ ]:
meta_data = compartment_report.meta()

cursor = conn.cursor()
conn.rollback()
sql_command = "DELETE FROM " + PGSCHEMA + ".simulation_report";
cursor.execute(sql_command, conn)
conn.commit()

guid = 0
start_time = 0.0
end_time = float(meta_data['end_time'])
time_step = float(meta_data['time_step'])
time_unit = meta_data['time_unit']
if time_unit == '':
    time_unit = 'ms'
num_frames = int(meta_data['num_frames'])

sql_command = "INSERT INTO " + PGSCHEMA + ".simulation_report VALUES (%d, %f, %f, '%s', %d)" %\
    (guid, time_step, end_time, time_unit, num_frames)
print(sql_command)
cursor.execute(sql_command, conn)
conn.commit()

In [ ]:
mapping = compartment_report.read_mapping()

In [ ]:
# Compartment report
import progressbar

print('Cleaning up existing simulation data')
cursor = conn.cursor()
conn.rollback()
sql_command = "DELETE FROM " + PGSCHEMA + ".simulation_compartments";
cursor.execute(sql_command, conn)
conn.commit()

nb_frames = compartment_report.num_frames()
nb_cells = len(compartment_report.cell_gids())
print('Importing ' + str(nb_cells) + ' cells')

bar = progressbar.ProgressBar(max_value=nb_cells)

count = 0
for gid in compartment_report.cell_gids():
    bar.update(count)

    voltages = compartment_report.read_frame_range(start=0, stop=nb_frames, gids=[gid])
    values = list()
    for v1 in voltages[0]:
        for v2 in v1:
            values.append(v2)
            
    buffer = np.array(values, dtype=np.float32).tobytes()
    sql = "INSERT INTO " + PGSCHEMA + ".simulation_compartments VALUES (%s, %s)"
    cursor.execute(sql, (int(gid), psycopg2.Binary(buffer), ))
        
    conn.commit()
    count += 1
    
bar.finish()

## Somas

In [ ]:
import pybinreports
soma_report = pybinreports.BinReport(
    '$VISUALIZATION_FOLDER/simulations/vizCa2p0_1x7/soma.bbp')
mapping = soma_report.read_mapping()

In [ ]:
len(soma_report.gids())

In [ ]:
# Gids rows contain full simulation
conn.rollback()
cursor = conn.cursor()

sql = 'DELETE FROM simulation_somas'
cursor.execute(sql, conn)
conn.commit()

import progressbar
import array as arr
nb_frames = soma_report.num_frames()
gids=soma_report.gids()
nb_gids = len(gids)

bar = progressbar.ProgressBar(max_value=nb_gids)
for i in range(len(gids)):
    voltages = soma_report.read_frame_range(start=0, stop=nb_frames, gids=[gids[i]])
    
    b = arr.array('B')    
    for voltage in voltages[0]:
        b.append(voltage + 100)
    
    sql = "INSERT INTO simulation_somas VALUES (%s, %s)"
    cursor.execute(sql, (i, psycopg2.Binary(b), ))
    bar.update(i)
    conn.commit()
bar.finish()

In [ ]:
# Frame rows contain all guids

conn.rollback()
cursor = conn.cursor()

# sql = 'DELETE FROM simulation_somas_2'
# cursor.execute(sql, conn)
# conn.commit()

import progressbar
import array as arr
nb_frames = soma_report.num_frames()
gids=soma_report.gids()

bar = progressbar.ProgressBar(max_value=nb_frames)
for frame in range(12716, nb_frames):
    voltages = soma_report.read_frame_range(start=frame, stop=frame+1, gids=gids)
    b = arr.array('B')
    for voltage in voltages[0][0]:
        b.append(int(voltage) + 100)
    
    sql = "INSERT INTO simulation_somas_2 VALUES (%s, %s)"
    cursor.execute(sql, (frame, psycopg2.Binary(b), ))
    bar.update(frame)
    conn.commit()
bar.finish()

In [ ]:
# help(circuit.v2.morph.segment_points)
segments = circuit.v2.morph.segment_points(gid=10, neurite_type=bluepy.v2.morphology.NeuriteType.basal_dendrite)
# print(segments.keys())
for segment in segments:l
    print(segments.index.codes[section])
#     for segment in segments.index[index]:
#         print(segment)
#         for values in segments.loc[(index, segment)].get_values():
#             print(values)
#     break

## Compartments

In [ ]:
import pybinreports
compartment_report = pybinreports.BinReport(
    '$VISUALIZATION_FOLDER/simulations/vizCa2p0_1x7/compartments.bbp')

In [ ]:
mapping = compartment_report.read_mapping()

In [ ]:
# Compartment report
import progressbar

cursor = conn.cursor()
sql_command = "DELETE FROM simulation_compartments";
cursor.execute(sql_command, conn)
conn.commit()

nb_frames = compartment_report.num_frames()
nb_cells = len(compartment_report.cell_gids())

bar = progressbar.ProgressBar(max_value=nb_cells)

count = 0
for gid in compartment_report.cell_gids():
    
    voltages = compartment_report.read_frame_range(start=0, stop=nb_frames, gids=[gid])
    values = list()
    for v1 in voltages[0]:
        for v2 in v1:
            values.append(v2)
            
    buffer = np.array(values, dtype=np.float32).tobytes()
    sql = "INSERT INTO simulation_compartments VALUES (%s, %s)"
    cursor.execute(sql, (int(gid), psycopg2.Binary(buffer), ))
        
    count += 1
    bar.update(count)
    conn.commit()
bar.finish()

### Sections/Segments

In [ ]:
conn.rollback()
cursor = conn.cursor()

sql = 'DELETE FROM cell_segments'
cursor.execute(sql, conn)
conn.commit()

import progressbar
import array as arr
gids = circuit.v2.cells.ids()

count = 0
bar = progressbar.ProgressBar(max_value=len(gids))
for gid in gids:
    segments = circuit.v2.morph.segment_points(gid=gid, neurite_type=bluepy.v2.morphology.NeuriteType.all)
    indices = segments.index[0]
    for index in indices:
        print(index)
        b = arr.array('f')
        for values in segments.loc[index].get_values():
            for value in values:
                b.append(float(value))

        sql = "INSERT INTO cell_segments VALUES (%s, %s, %s, %s)"
        cursor.execute(sql % (int(gid), int(index), int(section_id), psycopg2.Binary(b), ))
    
        if count % 100 == 0:
            bar.update(count)
            conn.commit()
        count += 1
conn.commit()
bar.finish()

### Synapses

In [ ]:
circuit_config = 'BlueConfig'
circuit = bluepy.Circuit(circuit_config)
connectome = circuit.connectome
positions = connectome.synapse_positions
positions(side='pre', synapse_ids=[[10, 11]], kind='center')

In [ ]:
presynaptic_gids = circuit.get_presynaptic_gids(gid=1)

In [ ]:
conn.rollback()
cursor = conn.cursor()

sql = 'DELETE FROM synapse'
cursor.execute(sql, conn)
conn.commit()

import progressbar
import array as arr
gids = circuit.v2.cells.ids()

count = 0
bar = progressbar.ProgressBar(max_value=len(gids))
for gid in gids:
    presynaptic_gids = circuit.get_presynaptic_gids(gid=gid)
    postsynaptic_gids = circuit.get_postsynaptic_gids(gid=gid)
    sql = "INSERT INTO synapse VALUES (%s, %s, %s)"
    
    presyn = 'NULL'
    if presynaptic_gids is not None:
        presyn = str(presynaptic_gids.tolist())
        presyn = presyn.replace('[', '{')
        presyn = presyn.replace(']', '}')
        presyn = str("'" + presyn + "'")

    postsyn = 'NULL'
    if postsynaptic_gids is not None:
        postsyn = str(postsynaptic_gids.tolist())
        postsyn = postsyn.replace('[', '{')
        postsyn = postsyn.replace(']', '}')
        postsyn = str("'" + postsyn + "'")

    cursor.execute(sql % (int(gid), presyn, postsyn, ))
    bar.update(count)
    conn.commit()
    count += 1
bar.finish()

In [ ]:
circuit.get_presynaptic_data(gid=1)

In [ ]:
circuit.SYNAPTIC_DATA_MEANING

In [ ]:
conn.rollback()
cursor = conn.cursor()

sql = 'DELETE FROM synapse_data'
cursor.execute(sql, conn)
conn.commit()

import progressbar
import array as arr
guids = circuit.v2.cells.ids()

count = 0
bar = progressbar.ProgressBar(max_value=len(guids))
for guid in guids:

    synapse_id = 0
    data = circuit.get_presynaptic_data(gid=guid)
    if data is None:
        continue
    for d in data:
        connecting_guid=int(d[0])
        axonal_delay=float(d[1])
        post_section_guid=int(d[2])
        post_segment_guid=int(d[3])
        post_distance=float(d[4])
        pre_section_id=int(d[5])
        pre_segment_id=int(d[6])
        pre_distance=float(d[7])
        g_syn_x=float(d[8])
        u_syn=float(d[9])
        d_syn=float(d[10])
        f_syn=float(d[11])
        decay_time_constant=float(d[12])
        synapse_type=int(d[13])
        morphology_type=int(d[14])
        branch_order_dendrite=int(d[15])
        branch_order_axon=int(d[16])
        absolute_synaptic_efficacy=int(d[17])
        branch_type=int(d[18])

        sql = "INSERT INTO synapse_data VALUES ( \
            %i, %i, %d, \
            %i, %i, %d, \
            %i, %i, %d, \
            %d, %d, %d, \
            %d, %d, %i, \
            %i, %i, %i, \
            %i, %i, %i)"

        cursor.execute(sql % (
            guid,
            connecting_guid,
            axonal_delay,
            post_section_guid,
            post_segment_guid,
            post_distance,
            pre_section_id,
            pre_segment_id,
            pre_distance,
            g_syn_x,
            u_syn,
            d_syn,
            f_syn,
            decay_time_constant,
            synapse_type,
            morphology_type,
            branch_order_dendrite,
            branch_order_axon,
            absolute_synaptic_efficacy,
            branch_type,
            synapse_id))
        synapse_id += 1
    
    bar.update(count)
    conn.commit()
    count += 1
bar.finish()